# Jeu d'apprentissage *set-valued*

**<span style='color:blue'> Objectifs de la séquence</span>** 
* Être sensibilisé&nbsp;:
    * à l'idée de prédiction d'ensembles.
* Être capable&nbsp;:
    * d'implémenter un \textit{top-K} avec $\texttt{pytorch}$.



 ----
Soit $\mathcal{X}$ notre espace des données d'entrée et $\mathcal{Y}=\{1, \ldots, C\}$ notre espace des labels pour un problème de classification à $C$ classes. Le problème de classification multi-classes dans son *framework* traditionnel revient à chercher une application de $\mathcal{X}$ dans $\mathcal{Y}$.

Cependant, comme nous allons le voir, il est parfois souhaitable de prédire une liste de classes (un ensemble), plutôt qu'une classe indivuduelle. Dans cette première séquence, nous partons du principe qu'il existe un jeu de données d'apprentissage décrivant de tels ensembles.

## I. Présentation d'un jeu de données avec ambiguité

Le jeu de données d'annotation est téléchargeable à l'adresse suivante [CIFAR-10H](https://github.com/jcpeterson/cifar-10h/blob/master/data/cifar10h-counts.npy). Ce dossier a été construit en demandant à des gens d'annoter un jeu de données connu : "CIFAR-10". On s'est rendu compte suite à l'annotation que le jeu de données était entâché d'incertitude : chaque image peut correspondre à plusieurs labels. Observons cela.

In [ ]:
import numpy as np

A = np.load('cifar10h-counts.npy')

# construct proba
A = A/A.sum(axis=1)[:, None]

max_proba = A.max(axis=1)

In [ ]:
from torchvision import datasets

cifar_train = datasets.CIFAR10(root='./data', train=True, download=True)
cifar_test = datasets.CIFAR10(root='./data', train=False, download=True)

classes = cifar_test.classes
class_to_idx = cifar_test.class_to_idx

X, y = cifar_test.data, cifar_test.targets

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 8))
plt.suptitle('CIFAR-10 is ambiguous!')
for i, idx in enumerate(np.argsort(max_proba)[:4]):
    plt.subplot(2, 4, i+1)
    plt.imshow(X[idx])
    plt.axis('off')
    
for i, idx in enumerate(np.argsort(max_proba)[:4]):
    ax = plt.subplot(2, 4, i+5)
    labels_sorted = (-A[idx]).argsort()
    labels = [classes[j] for j in labels_sorted[:4]]
    probas = [A[idx][j] for j in labels_sorted[:4]]
    color = ['lightcoral' if j == y[idx] else 'lightblue' for j in labels_sorted[:4]]
    ax.bar(labels, probas, color=color)

plt.show()

La bonne classe (celle du jeu de données officiel) est mise en avant.

Un modèle qui retournerait dans ces cas de figure l'ensemble des classes serait-il un mauvais modèle ? *A priori* on voudrait dire que non.

## II. Un problème toujours multi-classes ?

Nous disposons donc d'un problème de classification multi-classe tel que $\mathcal{X}$ est l'espace de nos données et $\mathcal{Y}$ celui de nos labels. Cependant, notre objectif est de construire un modèle dont la prédiction se fait sur $\mathcal{P}(\mathcal{Y})$, soit l'ensemble des parties de $\mathcal{Y}$. Illustrons cela par un exemple. Si $\mathcal{Y}=\{0, 1\}$, alors $\mathcal{P}(\mathcal{Y})=\{\emptyset, \{0\}, \{1\}, \{0, 1\}\}$. C'est exactement ce qu'on veut. Finalement, on se rend compte que cela revient à considérer un nouveau problème de classification où les classes sont les éléments de $\mathcal{P}$. Ainsi, au lieu d'avoir un problème de classification binaire, nous aurions ici $4$ classes. Et si le modèle prédit la troisième classe, alors on retournerait $\{0, 1\}$.

Cette stratégie est malheureusement vouée à l'échec à moins que nous sachions à l'avant qu'une quantité limité d'ambiguïté est possible. En effet, si on note $|\mathcal{Y}|$ le cardinal de $\mathcal{Y}$, alors $|\mathcal{P}(\mathcal{Y})|=2^{|\mathcal{Y}|}$. Sur $\texttt{CIFAR-10}$, notre problème à 10 classes en possède maintenant $1024$. Il devient même très probable que certaines configurations n'aient jamais été vues (et ne possèdent donc aucun exemple d'apprentissage).

L'astuce consiste à considérer les classes comme des entités séparées et indépendantes. Une classe est possible indépendamment de son voisinage.

## III. Un problème multi-labels

Si l'information est disponible, alors il est possible de reformuler notre problème d'apprentissage multi-classes en un problème d'apprentissage multi-labels. Ici, l'espace de nos labels devient $\mathcal{Y}=\{0, 1\}^C$. Dit autrement chaque classe possible est, ou n'est pas acceptée. Notons que pour réfléchir de cette manière, il est absolument nécessaire que les données contiennent l'information d'ensemble (i.e. nous devons avoir un jeu de données d'apprentissage multi-labels). Nous devons pouvoir pénaliser notre modèle s'il retourne une classe dans notre ensemble alors qu'elle n'aurait pas du y être. Dans le cas contraire, il suffirait de retourner tout le temps toutes les classes pour être sûrs de retourner la bonne classe.

### A. Le classifieur de Bayes

**<span style='color:blue'> Question </span>** **Imaginons que nous disposions de la loi $\eta_k(x)=\mathbb{P}(Y_k=1|X=x)$ indiquant la probabilité que la classe $k$ soit présente pour notre image $x$. Quelle est la meilleure prédiction à faire. C'est le classifieur de Bayes.**


 ----

### B. Des estimateurs de la probabilité conditionnelle

Il suffit ainsi de considérer un modèle de classification binaire par classe. Nous allons ici développer des modèles de réseaux de neurones cherchant à estimer la probabilité conditionnelle $\hat{\eta}_k$. Comme nous l'avons vu lors de la séquence multi-tâches, partager l'apprentissage de notre modèle de réseaux de neurones entre plusieurs tâches permet d'améliorer l'apprentissage de représentation.

Tout d'abord construisons notre jeu de données multi-labels en considérant qu'un label est présent à partir du moment où sa probabilité suite à l'annotation humaine est supérieure à $5\%$.

In [ ]:
labels_test = (A > 0.05).astype(float)

In [ ]:
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

In [ ]:
transform = transforms.Compose(
  [
      transforms.ToTensor(),
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
  ]
)

cifar_train = datasets.CIFAR10(root='./data', train=True, transform=transform)
cifar_test = datasets.CIFAR10(root='./data', train=False, transform=transform)

# on indique les nouveaux labels
cifar_test.targets = labels_test

batch_size = 128

trainloader = DataLoader(
  cifar_train, batch_size=batch_size
)

testloader = DataLoader(
  cifar_test, batch_size=batch_size, shuffle=True
)

print('Nb test batchs:', len(testloader))

**<span style='color:blue'> Question</span>** **Devons-nous adapter le code du modèle suivant de manière à lui permettre de gérer un apprentissage multi-labels ? Si oui, adaptez-le.**


 ----

In [ ]:
####### Complete this part ######## or die ####################
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.fc1 = nn.Linear(32 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc(x)
        return x
###############################################################

In [ ]:
model = Net()

In [ ]:
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR

**<span style='color:blue'> Question</span>** **Devons-nous adapter le code de la *loss* de manière à lui permettre de gérer un apprentissage multi-labels ? Si oui, adaptez-le.**


 ----

In [ ]:
####### Complete this part ######## or die ####################
#Choose the loss function
criterion = nn.CrossEntropyLoss()

#Optimizer
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9, weight_decay=0.)
###############################################################


Notez que seul le test a été annoté par des êtres humains. Nous devons donc malheureusement inverser le jeu d'apprentissage et le jeu de test.

In [ ]:
loss_history = []
for epoch in range(4):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(testloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:  # print every 2000 mini-batches
            # TODO uncomment print('\r[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000), end='')
            loss_history.append(running_loss / 2000)
            running_loss = 0.0
print('\r**** Finished Training ****')
plt.figure(figsize=(12, 8))
plt.plot([i for i in range(1, len(loss_history)+1)], loss_history, 
         label='My set-valued model')
plt.legend()
plt.show()


**<span style='color:blue'> Exercice</span>** **Proposez une évaluation *set-valued* en utilisant l'attribut $\texttt{threshold}$. La méthode doit retourner le score moyen ainsi que la taille moyenne des ensembles.**


 ----

In [ ]:
def test(model, loader, threshold, timeout=5):
    model.eval()  # on passe le modele en mode evaluation
    correct = 0
    total = 0
    set_size = 0
    with torch.no_grad():
        for i, data in enumerate(loader):
            images, labels = data
            outputs = model(images)
            # TODO Correction a supprimer
            ####### Complete this part ######## or die ####################
            correct += ...
            set_size += ...
            ###############################################################
            total += labels.size(0)
            if i >= timeout:
                break

    model.train()  # on remet le modele en mode apprentissage
    return correct / total, set_size / total


In [ ]:
set_size = []
accuracy = []
for threshold in [-i for i in range(10)]:
    accu, size = test(model, trainloader, threshold)
    set_size.append(size)
    accuracy.append(accu)


In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(set_size, accuracy)
plt.xlabel('Set size')
plt.ylabel('Accuracy')
plt.show()


Certes notre modèle n'est pas très bon, mais il est important de noter que l'apprentissage a eu lieu sur le jeu de test qui est beaucoup plus petit que le jeu d'apprentissage. Cela est du au fait que labeliser un jeu d'apprentissage complet en multi-labels est très dur. Ce genre d'information n'est généralement pas disponible et nous devons agir autrement comme nous le verrons dans la séquence suivante.

Cependant, notons que dès qu'on accepte de retourner $2$ labels en moyenne, notre modèle atteint presque $70\%$ d'accuracy.